In [2]:
# LIBRERIAS

import os
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import math
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt



#Comprobar que Tensorflow reconoce la tarjeta gráfica del equipo
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Lectura de ficheros con los datasets de las empresas a analizar
companies = {}
ficheros = os.listdir('data')

for file in ficheros:
    company = file[:-4]
    dfcompany = pd.read_csv(f"data/{file}")
    companies[company] = dfcompany

In [4]:
# Comprobación que de todas las empresas se dispone de al menos 1000 días de datos
for company in companies:
    dfcompany = companies[company]
    if dfcompany.shape[0] < 1000:
        print(company)

In [5]:
# Crreación de una matriz de datos de para la red neuronal
def create_dataset(dataset, look_back):
	x_data, y_data = [], []
	for i in range(len(dataset)-look_back-1):
		x = dataset[i:(i+look_back), 0]
		x_data.append(x)
		y = dataset[i + look_back, 0]
		y_data.append(y)
        
	return np.array(x_data), np.array(y_data)

def graficar(dfcompany, company):
	plt.plot(dfcompany)
	plt.xlabel('Days')
	plt.ylabel('Price')
	plt.title(label=company)

# Graficar el resultado
def graficarResultado(dfcompany, scaler, look_back, train_predict, test_predict):
    # Ajustamos los datos para mostrar la predicción
    trainPredictPlot = np.empty_like(dfcompany)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[look_back:len(train_predict)+look_back,:] = train_predict
    
    testPredictPlot = np.empty_like(dfcompany)
    testPredictPlot[:, :] = np.nan
    testPredictPlot[len(train_predict)+(look_back*2)+1:len(dfcompany)-1, :] = test_predict

    # Mostrar las los datos originales y la predicción
    plt.plot(scaler.inverse_transform(dfcompany), label='Original data')
    plt.plot(trainPredictPlot, label='Train prediction')
    plt.plot(testPredictPlot, label='Test prediction')
    plt.legend()
    
    plt.xlabel('Days')
    plt.ylabel('Price')
    plt.show()
	
#Separar datos de entrenamiento y test
def split(dfcompany, look_back, percent=70):
    training_size = int(len(dfcompany)*percent) 
    test_size = len(dfcompany)-training_size 

    train_data, test_data = dfcompany[0:training_size,:], dfcompany[training_size:len(dfcompany),:1]


    #Creamos la matrix para los datos de entrenamiento y test    
    X_train, y_train = create_dataset(train_data, look_back) 
    X_test, y_test = create_dataset(test_data, look_back) 
    #print(f"La matriz X_train tiene la forma: {X_train.shape}")
    #print(f"La matriz y_train tiene la forma: {y_train.shape}")
    #print(f"La matriz X_test tiene la forma: {X_test.shape}")
    #print(f"La matriz y_test tiene la forma: {y_test.shape}")
    

    # Hay que hacer reshape a las entradas de la red LSTM añadiendo 1 dimension [samples, look_back, features]
    X_train = X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
    X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)


    return X_train, X_test, y_train, y_test

In [6]:
def model_builder(hp):
  '''
  Args:
    hp - Keras tuner object
  '''
  # Initialize the Sequential API and start stacking the layers
  model = keras.Sequential()

  # Tune the number of units in the first LSTM layer
  # Choose an optimal value between 10-100
  hp_units0 = hp.Int('units0', min_value=10, max_value=100, step=20)
  model.add(LSTM(units=hp_units0, input_shape=(look_back,1), return_sequences=True, name='LSTM1'))

  # Tune the number of units in the second LSTM layer
  # Choose an optimal value between 10-100
  hp_units1 = hp.Int('units1', min_value=10, max_value=100, step=20)
  model.add(LSTM(units=hp_units1, name='LSTM2'))
  
  # Add next layers
  model.add(keras.layers.Dropout(0.2))
  model.add(Dense(1, activation='linear'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mean_squared_error',
                metrics=['mse'])

  return model

In [7]:
def prediccion (model, X_train, X_test, y_train, y_test, look_back, scaler):
    # Realizar predicción sobre datos de entranamiento y test
    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)

    # Transformamos los datos a su escala original
    train_predict = scaler.inverse_transform(train_predict)
    test_predict = scaler.inverse_transform(test_predict)

    return  train_predict, test_predict

def generarHipermodelo(X_train, y_train, X_test, y_test): 


    # Perform hypertuning
    tuner.search(X_train, y_train, epochs=10, validation_data=(X_test,y_test), callbacks=[stop_early]) 

    best_hp = tuner.get_best_hyperparameters()[0]

    # Build the model with the optimal hyperparameters
    h_model = tuner.hypermodel.build(best_hp)
    h_model.summary()
    print(tuner.get_best_hyperparameters()[0].get('learning_rate'))

    return h_model

def generarResultados(dfcompany):
    #dfcompany = companies[company]
    # Por cada empresa, preprocesar, entrenar y guardar resultados
    company = dfcompany.iloc[0,0]

    # Descartamos todas las columnas y nos quedamos unicamente con el precio de cierre de la acción
    dfcompany = dfcompany.reset_index()['close']
    scaler = MinMaxScaler(feature_range=(0,1))
    dfcompany = scaler.fit_transform(np.array(dfcompany).reshape(-1,1))
    X_train, X_test, y_train, y_test = split(dfcompany, look_back, percent=0.90)

    h_model = generarHipermodelo(X_train, y_train, X_test, y_test)
    #Entrenamiento del modelo mejorado
    history = h_model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, callbacks=[stop_early])

    train_predict, test_predict = prediccion(h_model, X_train, X_test, y_train, y_test, look_back, scaler)

    # Calculamos el error en entrenamiento y test
    train_error = math.sqrt(mean_squared_error(y_train, train_predict))
    test_error = math.sqrt(mean_squared_error(y_test, test_predict))

    print(f"Para la empresa {company}")
    print(f"Error entrenamiento: {train_error}")
    print(f"Error test: {test_error}")

    #graficarResultado(dfcompany, scaler, look_back, train_predict, test_predict)
    return (train_error, test_error)

In [8]:
# Definimos cuantos días usará el modelo para predecir el día siguiente
look_back = 50

# Instanciamos el tuner
tuner = kt.Hyperband(model_builder, # the hypermodel
                    objective='val_loss', # objective to optimize
                    max_epochs=10,
                    factor=3, # the denominator of the number of models trained in every iteration of the training loop.
                    directory='keras_tuner_dir', # directory to save logs 
                    project_name='Stock_prediction')

# hypertuning settings
tuner.search_space_summary() 

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=10, min_delta=0.2)

INFO:tensorflow:Reloading Oracle from existing project keras_tuner_dir\Stock_prediction\oracle.json
INFO:tensorflow:Reloading Tuner from keras_tuner_dir\Stock_prediction\tuner0.json
Search space summary
Default search space size: 3
units0 (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 100, 'step': 20, 'sampling': None}
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 100, 'step': 20, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [9]:
dictErrores = {}
for company in companies:
    #if company == 'AMZN':
    print("\n\n\n")
    print(f"Entrenando para {company}...")
    dfcompany = companies[company]
    dictErrores[company] = generarResultados(dfcompany)





Entrenando para AAL...
INFO:tensorflow:Oracle triggered exit
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM1 (LSTM)                 (None, 50, 90)            33120     
_________________________________________________________________
LSTM2 (LSTM)                 (None, 30)                14520     
_________________________________________________________________
dropout_1 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 47,671
Trainable params: 47,671
Non-trainable params: 0
_________________________________________________________________
0.01
Epoch 1/50
14/14 [==============================] - 12s 115ms/step - loss: 0.1085 - mse: 0.1085 - val_loss: 0.0057 - val_mse: 0.0057
Epoch 2/50
14/14 [===========================

In [10]:
for company in dictErrores:
    print (company+ " " + str(dictErrores[company]))

AAL (31.986895876670655, 17.112231552858635)
AAPL (217.55420603051678, 173.71733273485268)
ADBE (362.12634433392094, 430.12670731828473)
ADI (121.54032627656188, 164.8940050484704)
ADP (152.22154278734675, 203.45308013621548)
ADSK (200.36757654717726, 204.3070353457006)
AKAM (87.81514978767088, 109.36767417175184)
ALXN (117.44982008201941, 167.90884098045626)
AMAT (77.78605838807883, 131.46767989698716)
AMGN (210.6008217202712, 238.28452561743333)
AMZN (2318.5072753271493, 2871.3693253831552)
ATVI (69.65319369000632, 78.31054705719026)
AVGO (332.4756415379994, 614.3274659968503)
BBBY (19.129598002691694, 17.86404704923165)
BIDU (181.53397949020976, 136.8116980587504)
BIIB (291.9154947465443, 209.9152267317921)
BMRN (87.54738779045249, 82.03921456147728)
CERN (68.51375804068878, 88.74550067557006)
CHKP (113.55338685926702, 133.1714753740786)
CHTR (507.10561621502933, 561.9437559338056)
CMCSA (43.412134778990016, 45.076490490375676)
COST (305.1411240373245, 542.210039290334)
CSCO (46.727

In [11]:
errors = pd.DataFrame(list(dictErrores.values()))
#print(errors)
train_errors = errors[0]
test_errors = errors[1]


print (f"La media del RMSE obtenido en entrenamiento entrenando cada empresa por separado es {np.mean(train_errors)}")
print (f"La media del RMSE obtenido en test entrenando cada empresa por separado es {np.mean(test_errors)}")

La media del RMSE obtenido en entrenamiento entrenando cada empresa por separado es 183.45011332251832
La media del RMSE obtenido en test entrenando cada empresa por separado es 223.46703801129905


In [12]:
df_train_errors = pd.DataFrame(train_errors)
df_test_errors = pd.DataFrame(test_errors)

#df_train_errors.describe()
df_test_errors.describe()

,1
count,89.000000
mean,223.467038
std,415.389260
min,5.703670
25%,56.060894
50%,107.926943
75%,204.307035
max,2871.369325
